# Import Packages








In [23]:
import os
import pandas as pd
import numpy as np
import scipy.optimize as sc 
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import Image
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler



# Import the Dataset


In [24]:
# Import data from files
os.chdir('/Users/Yiru/Desktop/Big Data/CW')
data = pd.read_csv("Returns_Data.csv")
data['date'] = pd.to_datetime(data['date'], dayfirst=False)
characteristics = pd.read_csv("Stock_Characteristics_Data.csv")
dictionary = pd.read_excel("StockDataDictionary.xlsx")

# Create new dataframes
prices = data.pivot(index='date', columns='ticker', values='PRC')
volume = data.pivot(index='date', columns='ticker', values='VOL')
returns = data.pivot(index='date', columns='ticker', values='RET')
#returns = returns * 100 # Scale returns to percentage

# Summary of Returns
returns_summary = returns.describe()
shares_outstanding = data.pivot(index='date', columns='ticker', values='SHROUT')
value_weighted_returns = data.pivot(index='date', columns='ticker', values='vwretd')
equal_weighted_returns = data.pivot(index='date', columns='ticker', values='ewretd')
tickers = prices.columns # List of Tickers


# Data Preparation
We consider use the Lasso Regression to find the portfolio, the characteristics in "Stock_Characteristics_Data" should be used as regressors X except the annual close price, and use the annual close price to calculate the annual returns, which is used as target variable y.

Besides, we noticed that there are many NaN values in the dataset, so we consider to use interpolation to estimate missing values based on the values of other data points. In addition, we also consider to use forward fill and backward fill to linear interpolation.

In [25]:
# Calculate the annual return
characteristics['annual_return'] = characteristics.groupby('ticker')['prcc_f'].pct_change().mul(100)
characteristics

#Use interpolation to estimate missing values
characteristics.interpolate(method='linear', inplace=True)
characteristics.ffill(inplace=True)
characteristics.bfill(inplace=True)


# LightGBM Regression

In [34]:

# Convert the 'date' column to datetime format
characteristics['datadate'] = pd.to_datetime(characteristics['datadate'])
characteristics['annual_return'] = pd.to_numeric(characteristics['annual_return'], errors='coerce')

train_start_date = '2009-12-31'
train_end_date = '2018-12-31'

test_start_date = '2019-01-01'
test_end_date = '2021-04-30'

# Filter the data based on the date ranges
train_set = characteristics[(characteristics['datadate'] >= train_start_date) & (characteristics['datadate'] <= train_end_date)]
test_set = characteristics[(characteristics['datadate'] >= test_start_date) & (characteristics['datadate'] <= test_end_date)]

# Separate features (X) and target variable (y) for training and test sets
X_train1 = train_set.drop(columns=['annual_return'])
y_train1 = train_set['annual_return']

X_test1 = test_set.drop(columns=['annual_return'])
y_test1 = test_set['annual_return']

#Make sure each variable are numerical data types
numeric_columns = characteristics.select_dtypes(exclude=['int64', 'object'])
characteristics_numeric = characteristics.select_dtypes(include=['float64', 'float32', 'int32', 'int16', 'uint8'])

X = characteristics_numeric.drop(columns=['annual_return'])
y = characteristics_numeric['annual_return']

#X_train, y_train, X_test and y_test data frame do not include the date and company name now
X_train = X_train1.select_dtypes(include=['float64', 'float32', 'int32', 'int16', 'uint8'])
y_train = pd.DataFrame({'annual_return': y_train1})
X_test = X_test1.select_dtypes(include=['float64', 'float32', 'int32', 'int16', 'uint8'])
y_test = pd.DataFrame({'annual_return': y_test1})

#check the scale of X_train 
scaler = StandardScaler()
X_scaled=scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# create the train/test data for LightGBM
lgb_train_data = lgb.Dataset(X_train_scaled, label=y_train)
lgb_test_data = lgb.Dataset(X_test_scaled, label=y_test)

# set the parameters
params = {
    'objective': 'regression',
    'metric': 'l2',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# train the model
num_round = 10
bst = lgb.train(params, lgb_train_data, num_round, valid_sets=[lgb_test_data])

# forecast the annual return
new_data_predictions = bst.predict(X_test_scaled, num_iteration=bst.best_iteration)

# assess the model
mse = mean_squared_error(y_test, new_data_predictions)
print('Mean Squared Error:', mse)


# Assign the predictions as a new column in the DataFrame
# Check the shapes of your arrays
print("X_test1 shape:", X_test1.shape)
print("new_data_predictions shape:", new_data_predictions.shape)

# Assuming you want to concatenate along axis 1 (columns)
min_rows = min(X_test1.shape[0], new_data_predictions.shape[0])

# Use integer indexing for slicing
X_test1_trimmed = X_test1[:min_rows]
new_data_predictions_trimmed = new_data_predictions[:min_rows]

# Assign the predictions as a new column in the DataFrame
X_test1_trimmed['new_data_predictions'] = new_data_predictions_trimmed

# print the updated data
print(X_test1_trimmed)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000918 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8999
[LightGBM] [Info] Number of data points in the train set: 960, number of used features: 37
[LightGBM] [Info] Start training from score 11.472234
Mean Squared Error: 633.5218824240583
X_test1 shape: (206, 42)
new_data_predictions shape: (240,)
       datadate ticker                        conm          act          at  \
10   2019-12-31    PNW  PINNACLE WEST CAPITAL CORP   1030.03000   18479.247   
11   2020-12-31    PNW  PINNACLE WEST CAPITAL CORP   1198.31900   20020.421   
22   2019-12-31    AXP         AMERICAN EXPRESS CO   1766.93848  198321.000   
23   2020-12-31    AXP         AMERICAN EXPRESS CO   1818.63116  191367.000   
34   2019-12-31    AFL                   AFLAC INC   2387.25064  152768.000   
...         ...    ...                         ...          ...         ...   
1175 202

C:\Users\Yiru\AppData\Local\Temp\ipykernel_14612\3792067800.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test1_trimmed['new_data_predictions'] = new_data_predictions_trimmed


# Portfolio optimization
There are nearly 100 firms in our dataset, and each company's stock returns can be regarded as a risky asset in our portfolio. Now we need to rearrange our data frame to compute the weights of each company more conveniently.


In [33]:
datadate = X_test1_trimmed['datadate']
datadate
ticker = X_test1_trimmed['ticker']
ticker
#X_test_scaled_pred_df=pd.DataFrame({'datadate':datadate},{'ticker':ticker},{'new_data_predictions': new_data_predictions})
#print(X_test_scaled_pred_df)

# Pivot the DataFrame
X_test_scaled_pivot_df = X_test1_trimmed.pivot(index="datadate",columns='ticker',values='new_data_predictions').reset_index()

# Display the result
print(X_test_scaled_pivot_df)









ticker   datadate        ADI        ADP       ADSK        AFL        AIZ  \
0      2019-01-31        NaN        NaN  12.566298        NaN        NaN   
1      2019-03-31        NaN        NaN        NaN        NaN        NaN   
2      2019-04-30        NaN        NaN        NaN        NaN        NaN   
3      2019-05-31        NaN        NaN        NaN        NaN        NaN   
4      2019-06-30        NaN   9.718025        NaN        NaN        NaN   
5      2019-07-31        NaN        NaN        NaN        NaN        NaN   
6      2019-08-31        NaN        NaN        NaN        NaN        NaN   
7      2019-09-30        NaN        NaN        NaN        NaN        NaN   
8      2019-10-31  12.993294        NaN        NaN        NaN        NaN   
9      2019-11-30        NaN        NaN        NaN        NaN        NaN   
10     2019-12-31        NaN        NaN        NaN   9.985558  -2.320854   
11     2020-01-31        NaN        NaN  14.620691        NaN        NaN   
12     2020-